<a href="https://colab.research.google.com/github/Vaycold/Python_ML/blob/main/Reco_Algo/Embedding%26Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goal :
  - Large-scale에서 적용 가능한 추천시스템 구조 이해
  - Word2vec 기반 임베딩
  - EDA에서 도출되는 다양한 INSIGHT
  - 